In [3]:
using Pkg
Pkg.activate("..")
using Revise
using FUSE

  Activating environment at `~/.julia/dev/FUSE/Project.toml`


In [5]:
@time data=FUSE.dd();

data.core_profiles.profiles_1d[1].grid.rho_tor_norm=range(0,1,length=10)
data.core_profiles.profiles_1d[1].electrons.density=(1.0.-data.core_profiles.profiles_1d[1].grid.rho_tor_norm).^2

cp = FUSE.core_profiles()
cp.profiles_1d[1].grid.rho_tor_norm=range(0,1,length=10)
cp.profiles_1d[1].electrons.density=(1.0.-cp.profiles_1d[1].grid.rho_tor_norm).^2

  0.000351 seconds (3.87 k allocations: 230.938 KiB)


10-element Vector{Float64}:
 1.0
 0.7901234567901234
 0.6049382716049383
 0.44444444444444453
 0.308641975308642
 0.19753086419753085
 0.11111111111111113
 0.04938271604938271
 0.01234567901234569
 0.0

In [ ]:
filename = joinpath(dirname(dirname(abspath(@__FILE__))), "sample", "sample_equilibrium_ods.json")
data  = FUSE.json2fuse(filename; verbose=true);

equilibrium
｜time → Vector{Float64}
｜ids_properties
｜｜comment → String
｜vacuum_toroidal_field
｜｜b0 → Vector{Float64}
｜｜r0 → Float64
｜time_slice
｜｜1
｜｜｜profiles_1d
｜｜｜｜psi → Vector{Float64}
｜｜｜｜f → Vector{Float64}
｜｜｜｜gm5 → Vector{Float64}
｜｜｜｜b_field_max → Vector{Float64}
｜｜｜｜dvolume_drho_tor → Vector{Float64}
｜｜｜｜gm9 → Vector{Float64}
｜｜｜｜dpsi_drho_tor → Vector{Float64}
｜｜｜｜f_df_dpsi → Vector{Float64}
｜｜｜｜surface → Vector{Float64}
｜｜｜｜r_outboard → Vector{Float64}
｜｜｜｜rho_tor → Vector{Float64}
｜｜｜｜gm8 → Vector{Float64}
｜｜｜｜j_tor → Vector{Float64}
｜｜｜｜dpressure_dpsi → Vector{Float64}
｜｜｜｜phi → Vector{Float64}
｜｜｜｜triangularity_upper → Vector{Float64}
｜｜｜｜b_field_min → Vector{Float64}
｜｜｜｜r_inboard → Vector{Float64}
｜｜｜｜darea_drho_tor → Vector{Float64}
｜｜｜｜q → Vector{Float64}
｜｜｜｜darea_dpsi → Vector{Float64}
｜｜｜｜squareness_lower_outer → Vector{Float64}
｜｜｜｜triangularity_lower → Vector{Float64}
｜｜｜｜pressure → Vector{Float64}
｜｜｜｜area → Vector{Float64}
｜｜｜｜squareness_upper_inner → Vector{F

In [7]:
epsilon = 1.8
St=1.0
Sn=1.0
Sj=1.0
effectiveZ=2.0

bootstrapCoefficient = FUSE.collisionless_bootstrap(FUSE.fuse_parameters[:bootstrapModel], epsilon, St, Sn, Sj, effectiveZ)
println(bootstrapCoefficient)

LoadError: ArgumentError: Package StructArrays not found in current path:
- Run `import Pkg; Pkg.add("StructArrays")` to install the StructArrays package.
